<a href="https://colab.research.google.com/github/torrhen/cable-temperature-prediction/blob/master/cable_temperature_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# data hosted in public repository
DATA_CSV_URL = "https://raw.githubusercontent.com/torrhen/cable-temperature-prediction/master/cable.csv"

In [2]:
import os
# create new project folder
os.makedirs('cable_temperature_prediction', exist_ok=True)

In [3]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## cable_temperature_prediction/data.py

In [4]:
#%writefile cable_temperature_prediction/data.py
import pandas as pd
import datetime as dt
import numpy as np
import torch

STANDARD_DEVIATION_OUTLIER_THRESHOLD = 2
ROUNDING_PRECISION = 3

# create dataframe from raw data taken from GitHub repository
def create_dataframe(url):
  # set the timestamp column as the index of the dataframe
  df = pd.read_csv(url, index_col=0, parse_dates=[0], infer_datetime_format=True)
  # resample data to ensure no missing timestamps
  df = df.resample("5min").mean()
  # replace all nan values using the last valid observation for every column
  df = df.pad() # pad() is equivalent to fillna(method="ffill")
  df = df.round(ROUNDING_PRECISION)
  # remove values outside 2 SD of the mean of each calendar month
  df = remove_outliers(df)
  
  return df

# group data by calendar month and replace values outside 2 SD of the mean with the last valid observed value for every column
def remove_outliers(df):
  # add new column string the integer month of each timestamp
  df["month"] = df.index.month

  # replace data points beyond 2 SD of the mean for each calendar month with np.nan
  def remove_data(group, std):
    group[np.abs(group - group.mean()) > std * group.std()] = np.nan
    return group

  transformed_df = df.groupby("month", as_index=False).transform(lambda x: remove_data(x, STANDARD_DEVIATION_OUTLIER_THRESHOLD))
  # fill empty data with the last valid observation for all columns
  transformed_df = transformed_df.fillna(method="ffill").round(ROUNDING_PRECISION)
  # remove integer month column
  df.drop(columns=["month"], inplace=True)

  return transformed_df


In [5]:
# create dataframe from data downloaded from GitHUb
df = create_dataframe(DATA_CSV_URL)

In [6]:
print(len(df))

315360


In [7]:
test_df = df[:20]
test_df
#a = test_df['Thermocouple 7'].shift(5)
#a

,Thermocouple 1,Thermocouple 2,Thermocouple 3,Thermocouple 4,Thermocouple 5,Thermocouple 6,Thermocouple 7,Load Current (Blue),Load Current (Yellow),Load Current (Red),Air Temperature
Timestamp,,,,,,,,,,,
2017-01-01 00:00:00,14.85,15.10,15.26,16.43,16.29,17.22,18.34,38.90,48.42,46.64,10.90
2017-01-01 00:05:00,14.85,15.09,15.26,16.42,16.29,17.21,18.34,39.14,48.80,47.02,10.88
2017-01-01 00:10:00,14.85,15.09,15.26,16.42,16.29,17.22,18.34,39.25,48.53,46.77,10.87
2017-01-01 00:15:00,14.84,15.09,15.26,16.42,16.28,17.20,18.34,38.71,46.77,45.48,10.86
2017-01-01 00:20:00,14.85,15.09,15.26,16.42,16.29,17.21,18.34,38.89,47.10,45.66,10.85
2017-01-01 00:25:00,14.82,15.07,15.23,16.39,16.26,17.19,18.31,38.84,47.19,45.10,10.81
2017-01-01 00:30:00,14.82,15.07,15.23,16.40,16.26,17.18,18.30,39.48,47.55,45.34,10.79
2017-01-01 00:35:00,14.82,15.07,15.23,16.39,16.27,17.18,18.31,39.48,46.87,44.93,10.78
2017-01-01 00:40:00,14.82,15.07,15.23,16.40,16.26,17.18,18.31,40.34,46.71,43.72,10.77


In [8]:
y = test_df.pop('Thermocouple 7')
y

Timestamp
2017-01-01 00:00:00    18.34
2017-01-01 00:05:00    18.34
2017-01-01 00:10:00    18.34
2017-01-01 00:15:00    18.34
2017-01-01 00:20:00    18.34
2017-01-01 00:25:00    18.31
2017-01-01 00:30:00    18.30
2017-01-01 00:35:00    18.31
2017-01-01 00:40:00    18.31
2017-01-01 00:45:00    18.30
2017-01-01 00:50:00    18.30
2017-01-01 00:55:00    18.30
2017-01-01 01:00:00    18.29
2017-01-01 01:05:00    18.31
2017-01-01 01:10:00    18.30
2017-01-01 01:15:00    18.29
2017-01-01 01:20:00    18.29
2017-01-01 01:25:00    18.29
2017-01-01 01:30:00    18.28
2017-01-01 01:35:00    18.28
Freq: 5T, Name: Thermocouple 7, dtype: float64

In [9]:
y = y[5:]
y

Timestamp
2017-01-01 00:25:00    18.31
2017-01-01 00:30:00    18.30
2017-01-01 00:35:00    18.31
2017-01-01 00:40:00    18.31
2017-01-01 00:45:00    18.30
2017-01-01 00:50:00    18.30
2017-01-01 00:55:00    18.30
2017-01-01 01:00:00    18.29
2017-01-01 01:05:00    18.31
2017-01-01 01:10:00    18.30
2017-01-01 01:15:00    18.29
2017-01-01 01:20:00    18.29
2017-01-01 01:25:00    18.29
2017-01-01 01:30:00    18.28
2017-01-01 01:35:00    18.28
Freq: 5T, Name: Thermocouple 7, dtype: float64

In [10]:
x = test_df
x

,Thermocouple 1,Thermocouple 2,Thermocouple 3,Thermocouple 4,Thermocouple 5,Thermocouple 6,Load Current (Blue),Load Current (Yellow),Load Current (Red),Air Temperature
Timestamp,,,,,,,,,,
2017-01-01 00:00:00,14.85,15.10,15.26,16.43,16.29,17.22,38.90,48.42,46.64,10.90
2017-01-01 00:05:00,14.85,15.09,15.26,16.42,16.29,17.21,39.14,48.80,47.02,10.88
2017-01-01 00:10:00,14.85,15.09,15.26,16.42,16.29,17.22,39.25,48.53,46.77,10.87
2017-01-01 00:15:00,14.84,15.09,15.26,16.42,16.28,17.20,38.71,46.77,45.48,10.86
2017-01-01 00:20:00,14.85,15.09,15.26,16.42,16.29,17.21,38.89,47.10,45.66,10.85
2017-01-01 00:25:00,14.82,15.07,15.23,16.39,16.26,17.19,38.84,47.19,45.10,10.81
2017-01-01 00:30:00,14.82,15.07,15.23,16.40,16.26,17.18,39.48,47.55,45.34,10.79
2017-01-01 00:35:00,14.82,15.07,15.23,16.39,16.27,17.18,39.48,46.87,44.93,10.78
2017-01-01 00:40:00,14.82,15.07,15.23,16.40,16.26,17.18,40.34,46.71,43.72,10.77


In [11]:
x = test_df[:-1]
x

,Thermocouple 1,Thermocouple 2,Thermocouple 3,Thermocouple 4,Thermocouple 5,Thermocouple 6,Load Current (Blue),Load Current (Yellow),Load Current (Red),Air Temperature
Timestamp,,,,,,,,,,
2017-01-01 00:00:00,14.85,15.10,15.26,16.43,16.29,17.22,38.90,48.42,46.64,10.90
2017-01-01 00:05:00,14.85,15.09,15.26,16.42,16.29,17.21,39.14,48.80,47.02,10.88
2017-01-01 00:10:00,14.85,15.09,15.26,16.42,16.29,17.22,39.25,48.53,46.77,10.87
2017-01-01 00:15:00,14.84,15.09,15.26,16.42,16.28,17.20,38.71,46.77,45.48,10.86
2017-01-01 00:20:00,14.85,15.09,15.26,16.42,16.29,17.21,38.89,47.10,45.66,10.85
2017-01-01 00:25:00,14.82,15.07,15.23,16.39,16.26,17.19,38.84,47.19,45.10,10.81
2017-01-01 00:30:00,14.82,15.07,15.23,16.40,16.26,17.18,39.48,47.55,45.34,10.79
2017-01-01 00:35:00,14.82,15.07,15.23,16.39,16.27,17.18,39.48,46.87,44.93,10.78
2017-01-01 00:40:00,14.82,15.07,15.23,16.40,16.26,17.18,40.34,46.71,43.72,10.77


In [12]:
print(len(x))
print(len(y))

19
15


In [13]:
from torch.utils.data import Dataset

class CableDataset(Dataset):
  def __init__(self, df, seq_len):
    self.dataframe = df
    self.seq_len = seq_len
    self.targets = self.create_targets(self.dataframe, self.seq_len)
    # self.dataframe no longer contains 'Thermocouple 7' column because of earlier pop() 
    self.data = self.create_data(self.dataframe, self.seq_len)

  # normalize features and create a sliding sequence window 
  def create_data(self, df, seq_len):
    # normalize columns of dataframe
    data_df = (df - df.min()) / (df.max() - df.min())
    # ignore the last input as the target one interval ahead is being predicted
    data_df = data_df[:-1]
    # create a sliding window tensor from the features dataframe
    data_tensor = self.create_sliding_window(data_df, seq_len)
    return data_tensor

  def create_targets(self, df, seq_len):
    # store target column
    targets_df = df.pop('Thermocouple 7')
    # ignore values with no valid corresponding sequence input
    targets_df = targets_df[seq_len:]
    # convert targets to single tensor
    targets_tensor = torch.from_numpy(targets_df.values).type(torch.float32)
    return targets_tensor

  # creating features dataset
  def create_sliding_window(self, df, seq_len):
    column_tensors = []
    # generate a sliding window for a single column from the features dataframe
    def generate_sliding_window(df_column, seq_len):
      window_data = []
      df_column.rolling(seq_len).apply((lambda x: window_data.append(torch.from_numpy(x.values)) or 0), raw=False)
      # concatenate all samples into a single tensor of size [m, seq_len]
      return torch.cat(window_data).reshape(-1, seq_len)

    # generate sliding window for every column in the dataframe
    for col in df.columns:
      tensor = generate_sliding_window(df[col], seq_len)
      column_tensors.append(tensor)

    # create a tensor of size [m, seq_len, n] containing data from entire dataframe
    return torch.cat(column_tensors, dim=1).reshape(-1, len(df.columns), seq_len).permute(0, 2, 1).type(torch.float32)

  # override
  def __len__(self):
    return len(self.data)

  # override
  def __getitem__(self, idx):
    data = self.data[idx]
    target = self.targets[idx]
    return data, target

In [14]:
train_df = df[:400]
print(len(train_df))

400


In [15]:
test_df = df[400:500]
print(len(test_df))

100


In [16]:
train_data = CableDataset(train_df, 16)
test_data = CableDataset(test_df, 16)

print(train_data.data.shape)
print(train_data.targets.shape)
print(test_data.data.shape)
print(test_data.targets.shape)
print(train_data[2][0])
print(train_data[2][1])

torch.Size([384, 16, 10])
torch.Size([384])
torch.Size([84, 16, 10])
torch.Size([84])
tensor([[1.0000, 0.9961, 1.0000, 0.9960, 1.0000, 1.0000, 0.4464, 0.5733, 0.5546,
         0.8417],
        [0.9962, 0.9961, 1.0000, 0.9960, 0.9963, 0.9933, 0.4314, 0.5281, 0.5205,
         0.8396],
        [1.0000, 0.9961, 1.0000, 0.9960, 1.0000, 0.9966, 0.4364, 0.5366, 0.5252,
         0.8375],
        [0.9886, 0.9883, 0.9883, 0.9842, 0.9889, 0.9899, 0.4350, 0.5389, 0.5104,
         0.8292],
        [0.9886, 0.9883, 0.9883, 0.9881, 0.9889, 0.9866, 0.4528, 0.5481, 0.5168,
         0.8250],
        [0.9886, 0.9883, 0.9883, 0.9842, 0.9926, 0.9866, 0.4528, 0.5307, 0.5059,
         0.8229],
        [0.9886, 0.9883, 0.9883, 0.9881, 0.9889, 0.9866, 0.4767, 0.5266, 0.4740,
         0.8208],
        [0.9886, 0.9883, 0.9844, 0.9842, 0.9889, 0.9866, 0.4369, 0.5299, 0.4769,
         0.8146],
        [0.9886, 0.9883, 0.9883, 0.9842, 0.9889, 0.9866, 0.4703, 0.5510, 0.5300,
         0.8125],
        [0.9848, 0.9844

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data, batch_size=32, shuffle=False)
test_dataloader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)

## cable_temperature_prediction/models.py

In [18]:
from torch import nn

# custom RNN model
class RecurrentNeuralNetwork(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers):
    # initialize base class
    super().__init__()
    # store class attributes
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.output_size = output_size
    # store model layers
    self.rnn = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers, nonlinearity='relu', batch_first=True)
    self.block = nn.Sequential(
        nn.Linear(in_features=hidden_size, out_features=hidden_size*7*7),
        nn.ReLU(),
        nn.Linear(in_features=hidden_size*7*7, out_features=1)
    )

  def forward(self, x):
    # store the num of batches from input tensor
    batch_size = x.shape[0]
    # initialize new hidden state for each new forward pass
    h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).requires_grad_().to(device)
    # detach hidden state from computational graph and calculate output
    out, hid = self.rnn(x, h0.detach())
    # resize output before passing it to fully connected layer
    out = self.block(out[:, -1, :])

    return out

In [19]:
!pip install torchinfo
from torchinfo import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# initialize custom model [num_features, hidden_size, output_size, num_layers] and allocate to device
model_0 = RecurrentNeuralNetwork(10, 16, 1, 1).to(device)
# print a summary of the model using a generic input shape
summary(model_0, input_size=(32, 100, 10), col_names=['input_size', 'output_size', 'trainable'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Trainable
RecurrentNeuralNetwork                   [32, 100, 10]             [32, 1]                   True
├─RNN: 1-1                               [32, 100, 10]             [32, 100, 16]             True
├─Sequential: 1-2                        [32, 16]                  [32, 1]                   True
│    └─Linear: 2-1                       [32, 16]                  [32, 784]                 True
│    └─ReLU: 2-2                         [32, 784]                 [32, 784]                 --
│    └─Linear: 2-3                       [32, 784]                 [32, 1]                   True
Total params: 14,561
Trainable params: 14,561
Non-trainable params: 0
Total mult-adds (M): 1.89
Input size (MB): 0.13
Forward/backward pass size (MB): 0.61
Params size (MB): 0.06
Estimated Total Size (MB): 0.80

## cable_temperature_prediction/train.py

In [22]:
# train step
def train_step(model, dataloader, loss_fn, optimizer, device):
  train_loss = 0.0
  # training mode
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    # allocate data to device
    X, y = X.to(device), y.to(device)
    # forward pass
    output = model(X)
    # calculate loss
    loss = loss_fn(output, y)
    train_loss += loss
    # prevent accumulation of gradients
    optimizer.zero_grad()
    # backpropagation
    loss.backward()
    # gradient descent update
    optimizer.step()<no docstring>
  
  train_loss /= len(dataloader)

  return train_loss

In [23]:
# test step
def test_step(model, dataloader, loss_fn, device):
  test_loss = 0.0
  # evaluation mode
  model.eval()
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      # allocate data to device
      X, y = X.to(device), y.to(device)
      # forward pass
      output = model(X)
      # calculate loss
      loss = loss_fn(output, y)
      test_loss += loss
      
    test_loss /= len(dataloader)
    #print(test_loss)
  
  return test_loss

In [24]:
# train and test model
def train(model, train_loader, test_loader, loss_fn, optimizer, device, epochs=10):
  # store results of model at each epoch
  results = {'train_loss':[], 'test_loss':[]}

  for epoch in range(epochs):
    train_loss = train_step(model, train_loader, loss_fn, optimizer, device)
    test_loss = test_step(model, test_loader, loss_fn, device)
    # display statistics at each epoch
    print(f"Epoch: {epoch + 1} | Train Loss: {train_loss:.5f} | Test Loss: {test_loss:.5f}")

    results['train_loss'].append(train_loss)
    results['test_loss'].append(test_loss)

  return results

In [25]:
# mean squared error loss function
loss_fn = nn.MSELoss()
# Adam optimizer
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=1e-2)

In [26]:
# run model and show results
model_0_results = train(model_0, train_dataloader, test_dataloader, loss_fn, optimizer, device, epochs=50)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1 | Train Loss: 117.34551 | Test Loss: 1.43661
Epoch: 2 | Train Loss: 6.59101 | Test Loss: 7.79220
Epoch: 3 | Train Loss: 3.44440 | Test Loss: 1.75266
Epoch: 4 | Train Loss: 1.96561 | Test Loss: 0.35746
Epoch: 5 | Train Loss: 5.73384 | Test Loss: 0.14712
Epoch: 6 | Train Loss: 3.16037 | Test Loss: 22.39676
Epoch: 7 | Train Loss: 6.34560 | Test Loss: 14.68071
Epoch: 8 | Train Loss: 2.77701 | Test Loss: 0.36665
Epoch: 9 | Train Loss: 2.62081 | Test Loss: 0.10945
Epoch: 10 | Train Loss: 3.85776 | Test Loss: 6.73101
Epoch: 11 | Train Loss: 0.66089 | Test Loss: 8.06402
Epoch: 12 | Train Loss: 0.66081 | Test Loss: 4.36103
Epoch: 13 | Train Loss: 0.34069 | Test Loss: 2.95841
Epoch: 14 | Train Loss: 0.38652 | Test Loss: 3.47017
Epoch: 15 | Train Loss: 0.38470 | Test Loss: 4.48449
Epoch: 16 | Train Loss: 0.33467 | Test Loss: 5.08998
Epoch: 17 | Train Loss: 0.33922 | Test Loss: 5.00171
Epoch: 18 | Train Loss: 0.32457 | Test Loss: 4.44822
Epoch: 19 | Train Loss: 0.30069 | Test Loss: 3.9201

In [27]:
# make predictions using the trained model on the test data
predictions = [] # store predictions
model_0.eval()
with torch.inference_mode():
  for X, y in test_dataloader:
    # allocate to device
    X, y = X.to(device), y.to(device)
    # calculate logits
    output = model_0(X)
    predictions.append(output.cpu())
    
# create tensor using list of predictions
y_predictions = torch.cat(predictions)

In [28]:
y_predictions.shape

torch.Size([84, 1])

In [29]:
y_predictions[:10]

tensor([[16.6540],
        [16.6881],
        [16.6059],
        [16.4884],
        [17.0406],
        [16.5446],
        [16.8706],
        [16.7841],
        [16.8864],
        [17.0565]])

In [30]:
test_data.targets[:10]

tensor([15.3600, 15.3600, 15.3700, 15.3800, 15.3800, 15.3900, 15.4000, 15.3900,
        15.4000, 15.4100])

In [31]:
# import matplotlib.pyplot as plt
# from cable_temperature_prediction import data

# df = data.create_dataframe(DATA_CSV_URL)

# # plot the data before and after the removal of outliers for a specific column
# column_name = 'Load Current (Red)'
# fig, ax = plt.subplots(2, 1, figsize=(20,5), sharex=True, sharey=True)
# df[column_name].plot(ax=ax[0], kind='line', alpha=0.8)
# ax[0].set_title("Before removal of outliers")
# tr_df = data.remove_outliers(df)
# tr_df[column_name].plot(ax=ax[1], kind='line', alpha=0.8)
# ax[1].set_title("After removal of outliers")
# plt.show()